##### Imports

In [4]:
import warnings
import pandas as pd
warnings.filterwarnings('ignore')

pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 1000)

In [5]:
%matplotlib inline

import numpy as np
import pandas as pd
from sklearn.feature_selection \
        import mutual_info_regression
        
from scipy.stats import spearmanr
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

In [6]:
from pathlib import Path
import pandas as pd
from filelock import FileLock
import dask.dataframe as dd

# Define the path
DATA_STORE = Path('/home/sayem/Desktop/Project/data/assets.h5')
lock_path = "/tmp/assets_h5_file.lock"  # Lock path

# Lock the file and retrieve the data
with FileLock(lock_path):
    # Use Dask to directly read the HDF5 files
    alpha_101_data_full = dd.read_hdf(DATA_STORE, key='factors/alpha_101').compute()
    common_data_full = dd.read_hdf(DATA_STORE, key='factors/common').compute()
    # with pd.HDFStore(DATA_STORE) as store:
    #     alpha_101_data_full = store['factors/alpha_101']
    #     common_data_full = store['factors/common']

# Filter the data using pandas operations
end_date = '2019-12-31'
start_date = '2018-01-01'

alpha_101_data = alpha_101_data_full.loc[
    (alpha_101_data_full.index.get_level_values("date") >= start_date) &
    (alpha_101_data_full.index.get_level_values("date") <= end_date)
]

common_data = common_data_full.loc[
    (common_data_full.index.get_level_values("date") >= start_date) &
    (common_data_full.index.get_level_values("date") <= end_date)
]


del alpha_101_data_full
del common_data_full

In [7]:
# Prepare an empty list to collect processed chunks
processed_data_list = []

# Given that `common_data` is the larger DataFrame, we'll chunk through that.
# Define a chunk size based on your system's memory; you can adjust it as needed.
CHUNK_SIZE = 100000

# Create function to chunk through DataFrame
def chunker(seq, size):
    return (seq[pos: pos + size] for pos in range(0, len(seq), size))

# Chunk through `common_data` based on the index
for start in range(0, common_data.shape[0], CHUNK_SIZE):
    end = start + CHUNK_SIZE
    
    # Selecting the current chunk
    common_data_chunk = common_data.iloc[start:end]
    
    # Extract unique tickers and dates from the current chunk
    tickers_in_chunk = common_data_chunk.index.get_level_values('ticker').unique()
    dates_in_chunk = common_data_chunk.index.get_level_values('date').unique()
    
    # Filter alpha_101_data based on tickers and dates in the current chunk
    filtered_alpha_101_data = alpha_101_data[
        alpha_101_data.index.get_level_values('ticker').isin(tickers_in_chunk) &
        alpha_101_data.index.get_level_values('date').isin(dates_in_chunk)
    ]
    
    # Merge current chunk with filtered alpha_101_data
    merged_chunk = common_data_chunk.merge(
        filtered_alpha_101_data, left_index=True, right_index=True, how='inner', suffixes=('', '_y')
    )

    # Drop any columns that end with "_y" suffix and "_x" (just in case, though "_x" might not appear in this context)
    merged_chunk = merged_chunk.drop(columns=[col for col in merged_chunk if col.endswith(('_y', '_x'))])

    processed_data_list.append(merged_chunk)

# Concatenate all processed chunks
final_data = pd.concat(processed_data_list)

print(f"Shape of the final combined data: {final_data.shape}")
print("Processing completed.")

del processed_data_list

Shape of the final combined data: (4191516, 624)
Processing completed.


In [8]:
import pandas as pd

# 1. Capitalize all column names
final_data.columns = [col.upper() for \
    col in final_data.columns]

In [9]:
final_data.info()

<class 'pandas.core.frame.DataFrame'>
MultiIndex: 4191516 entries, ('AA', Timestamp('2020-01-02 00:00:00')) to ('ZTS', Timestamp('2022-12-30 00:00:00'))
Columns: 624 entries, OPEN to ALPHA_101
dtypes: float32(384), float64(33), int32(198), int8(9)
memory usage: 10.2+ GB


In [10]:
final_data

OPEN        HIGH         LOW       CLOSE  \
ticker date                                                         
AA     2020-01-02   21.621901   21.651699   21.113199   21.215500   
       2020-01-03   20.969101   21.404400   20.949200   21.295000   
       2020-01-06   21.136900   21.245300   20.700701   20.800100   
       2020-01-07   20.859699   21.339701   20.790100   21.118200   
       2020-01-08   21.098200   21.185699   20.206800   20.264500   
...                       ...         ...         ...         ...   
ZTS    2022-12-23  144.509995  145.891998  143.539993  145.759995   
       2022-12-27  145.910004  146.147400  143.570007  145.300003   
       2022-12-28  145.179993  146.639999  143.770004  143.830002   
       2022-12-29  145.199997  148.509995  145.137802  148.149994   
       2022-12-30  147.199997  147.789993  144.740005  146.550003   

                         VOLUME    MARKET_CAP  SECTOR  RET_FRAC_ORDER  \
ticker date                                                             
AA     2020-01-02  3.091952e+06  5.703249e+09     4.0        3.379249   
       2020-01-03  3.040772e+06  5.703249e+09     4.0        3.469475   
       2020-01-06  3.089822e+06  5.703249e+09     4.0        2.957221   
       2020-01-07  5.035948e+06  5.703249e+09     4.0        3.388488   
       2020-01-08  3.746568e+06  5.703249e+09     4.0        2.499384   
...                         ...           ...     ...             ...   
ZTS    2022-12-23  1.017856e+06  8.818486e+10     2.0        9.503230   
       2022-12-27  9.578710e+05  8.818486e+10     2.0        8.846587   
       2022-12-28  1.443916e+06  8.818486e+10     2.0        7.550820   
       2022-12-29  1.298851e+06  8.818486e+10     2.0       12.513685   
       2022-12-30  1.249527e+06  8.818486e+10     2.0        9.542715   

                    RET_01D   RET_02D   RET_03D   RET_04D   RET_05D   RET_10D  \
ticker date                                                                     
AA     2020-01-02 -0.004196  0.003719 -0.007434 -0.001403 -0.007434  0.034252   
       2020-01-03  0.003747 -0.000465  0.007480 -0.003715  0.002339  0.026245   
       2020-01-06 -0.023240 -0.019580 -0.023694 -0.015934 -0.026869 -0.021871   
       2020-01-07  0.015293 -0.008302 -0.004586 -0.008763 -0.000885  0.004261   
       2020-01-08 -0.040425 -0.025750 -0.048392 -0.044826 -0.048834 -0.047502   
...                     ...       ...       ...       ...       ...       ...   
ZTS    2022-12-23  0.005033  0.005796  0.023093  0.017948  0.008999 -0.049742   
       2022-12-27 -0.003156  0.001862  0.002622  0.019864  0.014736 -0.049457   
       2022-12-28 -0.010117 -0.013241 -0.008274 -0.007521  0.009546 -0.071885   
       2022-12-29  0.030035  0.019615  0.016397  0.021513  0.022288 -0.023916   
       2022-12-30 -0.010800  0.018911  0.008603  0.005420  0.010481 -0.014525   

                    RET_21D   RET_42D   RET_63D  RET_126D  RET_252D  \
ticker date                                                           
AA     2020-01-02  0.053542  0.030259  0.109789 -0.056439 -0.183721   
       2020-01-03  0.086391 -0.016928  0.128619 -0.070526 -0.180662   
       2020-01-06  0.062759 -0.083385  0.079639 -0.058265 -0.259002   
       2020-01-07  0.073019 -0.056178  0.107579 -0.040496 -0.249792   
       2020-01-08 -0.006817 -0.065786  0.106461 -0.095137 -0.275774   
...                     ...       ...       ...       ...       ...   
ZTS    2022-12-23 -0.031302 -0.040908 -0.017028 -0.157042 -0.394546   
       2022-12-27 -0.031979 -0.039961 -0.025375 -0.142510 -0.406236   
       2022-12-28 -0.029029 -0.046347 -0.053358 -0.169731 -0.406811   
       2022-12-29  0.002029 -0.031414 -0.016464 -0.134693 -0.395868   
       2022-12-30 -0.049241 -0.028054 -0.009637 -0.155438 -0.397843   

                   RET_FWD_FRAC_ORDER  RET_FWD_01D  RET_FWD_02D  RET_FWD_03D  \
ticker date                                                                    
AA     2020-01-02            3.469475  

In [11]:
import pandas as pd


# Extract min and max dates
min_date = final_data.index.get_level_values('date').min().strftime('%Y-%m-%d')
max_date = final_data.index.get_level_values('date').max().strftime('%Y-%m-%d')

# Define file path
FILE_PATH = "/home/sayem/Desktop/Project/data/dataset.h5"

# Use with statement for better memory management
with pd.HDFStore(FILE_PATH, \
    mode='a', complevel=9, complib='zlib') as store:
    final_data.to_hdf(store, \
        key=f'data/{min_date}_to_{max_date}', format='table')
    print(f"Data saved to {FILE_PATH}")
    print(f'key is data/{min_date}_to_{max_date}')

# print(f"Data saved to {FILE_PATH}")

Data saved to /home/sayem/Desktop/Project/data/dataset.h5
key is data/2020-01-02_to_2022-12-30


In [13]:
import pandas as pd


# Extract min and max dates
min_date = subset_result.index.get_level_values('date').min().strftime('%Y-%m-%d')
max_date = subset_result.index.get_level_values('date').max().strftime('%Y-%m-%d')

# Define file path
FILE_PATH = "/home/sayem/Desktop/Project/data/dataset.h5"

# Use with statement for better memory management
with pd.HDFStore(FILE_PATH, \
    mode='a', complevel=9, complib='zlib') as store:
    subset_result.to_hdf(store, \
        key=f'data/{min_date}_to_{max_date}', format='table')
    print(f"Data saved to {FILE_PATH}")
    print(f'key is data/reduced_{min_date}_to_{max_date}')

Data saved to /home/sayem/Desktop/Project/data/dataset.h5
key is data/reduced_2022-07-21_to_2022-07-29


In [14]:
from utils import clear_large_vars
clear_large_vars()

In [15]:
import pandas as pd

# Define the file path
FILE_PATH = "/home/sayem/Desktop/Project/data/dataset.h5"

# Open the HDF5 file and get the keys
with pd.HDFStore(FILE_PATH) as store:
    keys = list(store.keys())

# Print the keys
for key in keys:
    print(key)

/data/2020-01-02_to_2022-12-30
/data/reduced_2022-07-21_to_2022-07-29


: 